In [125]:

import pyspedas
from pytplot import tplot, get_data
from pyspedas import tinterp
from pyspedas.analysis.tvectot import tvectot
import matplotlib.pyplot as plt

import xarray as xr
import polars as pl
import hvplot.polars

import scipy.stats
import dcor

import astropy.units as u


In [110]:
from datetimerange import DateTimeRange
from sunpy.time import TimeRange
from numpy import timedelta64
from datetime import datetime, timedelta

In [111]:
start = '2019-04-06T12:00'
end = '2019-04-07T12:00'

earth_start = '2019-04-09'
earth_end = '2019-04-14'

In [112]:
psp_timerange = TimeRange(start, end)

timerange_earth = TimeRange(earth_start, earth_end)

In [115]:

def validate(timerange):
    if isinstance(timerange, DateTimeRange):
        return [timerange.get_start_time_str(), timerange.get_end_time_str()]
    if isinstance(timerange, TimeRange):
        return [timerange.start.to_string(), timerange.end.to_string()]

## Estimate of arrival time

In [116]:
psp_spi_vars = pyspedas.psp.spi(trange=validate(psp_timerange), time_clip=True)

r_psp = get_data('psp_spi_SUN_DIST', xarray=True)
r_psp = pl.DataFrame(r_psp.to_dataframe().reset_index())

29-Jan-24 14:35:26: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spi/l3/spi_sf00_l3_mom/2019/


Using LEVEL=L3


29-Jan-24 14:35:26: File is current: /Users/zijin/data/psp/sweap/spi/l3/spi_sf00_l3_mom/2019/psp_swp_spi_sf00_l3_mom_20190406_v04.cdf
29-Jan-24 14:35:26: File is current: /Users/zijin/data/psp/sweap/spi/l3/spi_sf00_l3_mom/2019/psp_swp_spi_sf00_l3_mom_20190407_v04.cdf


In [117]:
km2au = u.km.to(u.AU)
r_earth = 1 * u.AU.to(u.km)
v_sw_slow = 250 * u.km / u.s
v_sw_fast = 500 * u.km / u.s

r_psp.with_columns(
    distance2sun = pl.col('psp_spi_SUN_DIST'),
    distance2earth = r_earth - pl.col('psp_spi_SUN_DIST')
).with_columns(
    dt2arrival_slow = pl.duration(seconds = pl.col('distance2earth') / v_sw_slow),
    dt2arrival_fast = pl.duration(seconds = pl.col('distance2earth') / v_sw_fast)
).with_columns(
    time2arrival_slow = pl.col('time') + pl.col('dt2arrival_slow'),
    time2arrival_fast = pl.col('time') + pl.col('dt2arrival_fast')
).with_columns(
    distance2sun = pl.col('distance2sun') * km2au,
    distance2earth = pl.col('distance2earth') * km2au
)

time,psp_spi_SUN_DIST,distance2sun,distance2earth,dt2arrival_slow,dt2arrival_fast,time2arrival_slow,time2arrival_fast
datetime[ns],f64,f64,f64,duration[μs],duration[μs],datetime[μs],datetime[μs]
2019-04-06 12:00:01.807751,2.6128e7,0.174658,0.825342,5d 17h 11m 17s,2d 20h 35m 38s,2019-04-12 05:11:18.807751,2019-04-09 08:35:39.807751
2019-04-06 12:00:08.798291,2.6129e7,0.174659,0.825341,5d 17h 11m 17s,2d 20h 35m 38s,2019-04-12 05:11:25.798291,2019-04-09 08:35:46.798291
2019-04-06 12:00:15.788832,2.6129e7,0.17466,0.82534,5d 17h 11m 16s,2d 20h 35m 38s,2019-04-12 05:11:31.788832,2019-04-09 08:35:53.788832
2019-04-06 12:00:22.779372,2.6129e7,0.17466,0.82534,5d 17h 11m 16s,2d 20h 35m 38s,2019-04-12 05:11:38.779372,2019-04-09 08:36:00.779372
2019-04-06 12:00:29.769853,2.6129e7,0.174661,0.825339,5d 17h 11m 15s,2d 20h 35m 37s,2019-04-12 05:11:44.769853,2019-04-09 08:36:06.769853
2019-04-06 12:00:36.760393,2.6129e7,0.174662,0.825338,5d 17h 11m 15s,2d 20h 35m 37s,2019-04-12 05:11:51.760393,2019-04-09 08:36:13.760393
2019-04-06 12:00:43.750933,2.6129e7,0.174663,0.825337,5d 17h 11m 14s,2d 20h 35m 37s,2019-04-12 05:11:57.750933,2019-04-09 08:36:20.750933
2019-04-06 12:00:50.741454,2.6129e7,0.174664,0.825336,5d 17h 11m 14s,2d 20h 35m 37s,2019-04-12 05:12:04.741454,2019-04-09 08:36:27.741454
2019-04-06 12:00:57.731934,2.6129e7,0.174665,0.825335,5d 17h 11m 13s,2d 20h 35m 36s,2019-04-12 05:12:10.731934,2019-04-09 08:36:33.731934


## Get velocity

In [118]:
psp_spi_vars = pyspedas.psp.spi(trange=validate(psp_timerange), time_clip=True)
swe_vars = pyspedas.ace.swe(trange=validate(timerange_earth), datatype = 'h0')

29-Jan-24 14:35:31: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/psp/sweap/spi/l3/spi_sf00_l3_mom/2019/


Using LEVEL=L3


29-Jan-24 14:35:31: File is current: /Users/zijin/data/psp/sweap/spi/l3/spi_sf00_l3_mom/2019/psp_swp_spi_sf00_l3_mom_20190406_v04.cdf
29-Jan-24 14:35:31: File is current: /Users/zijin/data/psp/sweap/spi/l3/spi_sf00_l3_mom/2019/psp_swp_spi_sf00_l3_mom_20190407_v04.cdf
29-Jan-24 14:35:32: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/ace/swepam/level_2_cdaweb/swe_h0/2019/
29-Jan-24 14:35:32: File is current: /Users/zijin/data/ace/swepam/level_2_cdaweb/swe_h0/2019/ac_h0_swe_20190409_v11.cdf
29-Jan-24 14:35:33: File is current: /Users/zijin/data/ace/swepam/level_2_cdaweb/swe_h0/2019/ac_h0_swe_20190410_v11.cdf
29-Jan-24 14:35:33: File is current: /Users/zijin/data/ace/swepam/level_2_cdaweb/swe_h0/2019/ac_h0_swe_20190411_v11.cdf
29-Jan-24 14:35:33: File is current: /Users/zijin/data/ace/swepam/level_2_cdaweb/swe_h0/2019/ac_h0_swe_20190412_v11.cdf
29-Jan-24 14:35:33: File is current: /Users/zijin/data/ace/swepam/level_2_cdaweb/swe_h0/2019/ac_h0_swe_20190413_v11.cdf


In [119]:
tvar = tvectot("psp_spi_VEL_RTN_SUN")
psp_ion_speed: xr.DataArray = get_data(tvar, xarray=True)

ace_ion_speed: xr.DataArray = get_data("Vp", xarray=True)

In [120]:
def calc_correlation(
    v1: xr.DataArray,
    v2: xr.DataArray,
    v1_timerange: TimeRange,
    v2_timerange: TimeRange,
    cadence: timedelta = timedelta(minutes=10),
    window: timedelta = timedelta(hours=1),
):
    v1 = v1.dropna("time").sel(time=slice(v1_timerange.start.to_datetime(), v1_timerange.end.to_datetime()))
    v2 = v2.dropna("time").sel(time=slice(v2_timerange.start.to_datetime(), v2_timerange.end.to_datetime()))
    v1_start = v1_timerange.start.to_datetime()
    v1_end = v1_timerange.end.to_datetime()
    
    results = []
    for temp_tr in v2_timerange.window(cadence=cadence, window=window):
        v2_temp_start = temp_tr.start.to_datetime()
        v2_temp_end = temp_tr.end.to_datetime()
        
        v2_temp = v2.sel(time = slice(v2_temp_start, v2_temp_end))
        v2_temp["time"] = v2_temp["time"] - timedelta64(v2_temp_start - v1_start)
        v1_temp = v1.interp_like(v2_temp)

        pearsonr = scipy.stats.pearsonr(v1_temp, v2_temp).statistic
        distance_correlation = dcor.distance_correlation(v1_temp, v2_temp)
        results.append([v1_start, v1_end, v2_temp_start, v2_temp_end, pearsonr, distance_correlation])

    return pl.DataFrame(results, schema=["v1_start", "v1_end", "v2_start", "v2_end", "Pearson correlation", "Distance correlation"])


29-Jan-24 14:35:35: /Users/zijin/micromamba/envs/psp_conjunction/lib/python3.11/site-packages/dcor/_fast_dcov_avl.py:554: UserWarning: Falling back to uncompiled AVL fast distance covariance terms because of TypeError exception raised: No matching definition for argument type(s) array(float64, 1d, C), array(float32, 1d, C), bool. Rembember: only floating point values can be used in the compiled implementations.
  warnings.warn(



In [ ]:

window = (psp_timerange.end - psp_timerange.start)
cadence = timedelta(minutes=10)

df = calc_correlation(
    psp_ion_speed,
    ace_ion_speed,
    psp_timerange,
    timerange_earth,
    cadence = cadence,
    window = window,
)

In [129]:
df

v1_start,v1_end,v2_start,v2_end,Pearson correlation,Distance correlation
datetime[μs],datetime[μs],datetime[μs],datetime[μs],f64,f64
2019-04-06 12:00:00,2019-04-07 12:00:00,2019-04-09 00:00:00,2019-04-10 00:00:00,0.129733,0.295187
2019-04-06 12:00:00,2019-04-07 12:00:00,2019-04-09 00:10:00,2019-04-10 00:10:00,0.115776,0.290461
2019-04-06 12:00:00,2019-04-07 12:00:00,2019-04-09 00:20:00,2019-04-10 00:20:00,0.125829,0.295799
2019-04-06 12:00:00,2019-04-07 12:00:00,2019-04-09 00:30:00,2019-04-10 00:30:00,0.134056,0.299338
2019-04-06 12:00:00,2019-04-07 12:00:00,2019-04-09 00:40:00,2019-04-10 00:40:00,0.141806,0.309491
2019-04-06 12:00:00,2019-04-07 12:00:00,2019-04-09 00:50:00,2019-04-10 00:50:00,0.137222,0.310114
2019-04-06 12:00:00,2019-04-07 12:00:00,2019-04-09 01:00:00,2019-04-10 01:00:00,0.130128,0.307473
2019-04-06 12:00:00,2019-04-07 12:00:00,2019-04-09 01:10:00,2019-04-10 01:10:00,0.132468,0.314047
2019-04-06 12:00:00,2019-04-07 12:00:00,2019-04-09 01:20:00,2019-04-10 01:20:00,0.137969,0.310688


In [128]:
df.hvplot('v2_start', ['Pearson correlation', 'Distance correlation'])

:NdOverlay   [Variable]
   :Curve   [v2_start]   (value)

In [162]:
scipy.stats.pearsonr(v1, v2).statistic
dcor.distance_correlation(v1, v2)


0.4751159337791697

In [34]:
ace_ion_speed

<xarray.DataArray 'Vp' (time: 864)>
array([ 520.72,  517.02,  516.28,  514.83,  514.23,  511.74,  511.51,
        513.92,  528.33,  510.62,  521.24,  518.9 ,  506.36, -999.9 ,
        506.93,  517.98,  525.03,  523.32,  521.32,  521.98,  525.89,
        528.58,  523.24,  524.99,  526.78,  517.85,  511.2 ,  508.91,
        506.08,  519.67,  527.56,  526.8 ,  530.19,  529.09,  526.76,
        525.41,  526.25,  524.78,  524.98,  516.29,  508.49,  510.31,
        502.32,  503.62,  495.54,  493.53,  494.45,  493.14,  491.76,
        496.91,  492.53,  492.43,  502.16,  506.75,  513.49,  511.88,
        498.09,  492.75,  488.8 ,  480.81,  478.92,  480.08,  481.89,
        483.52,  488.3 ,  517.84,  513.12,  495.18,  486.7 ,  487.59,
        492.02,  489.42,  474.2 ,  489.88,  479.56,  483.25,  479.94,
        482.18,  475.22,  468.93,  468.26,  466.95,  467.15,  468.44,
        467.  ,  464.41,  462.14,  463.23,  465.79,  466.55,  467.34,
        472.34,  472.36,  467.9 ,  470.16,  464.12,  473.78,  477.88,
        479.3 ,  480.3 ,  473.14,  465.25,  463.96,  461.95,  465.84,
        469.93,  471.68,  475.68,  474.26,  464.34,  464.35,  465.09,
        469.99,  466.79,  460.61,  459.87,  460.24,  461.94,  465.63,
        467.62,  463.9 ,  469.23,  470.56,  461.88,  461.8 ,  459.67,
        457.79,  461.1 ,  465.22,  458.88,  459.5 ,  456.99,  462.77,
        456.98,  452.85,  453.86,  456.36,  457.44,  458.74,  463.45,
...
        452.27,  458.62,  474.37,  476.36,  467.43,  461.47,  467.7 ,
        467.01,  463.38,  466.67,  467.05,  474.61,  474.97,  453.35,
        448.53,  446.06,  463.11,  464.44,  449.11,  459.06,  459.79,
        446.83,  446.64,  444.66,  441.74,  442.02,  438.87,  438.69,
        445.03,  452.8 ,  456.41,  459.54,  442.53,  442.67,  443.96,
        443.91,  439.28,  437.85,  438.41,  443.75,  445.95,  452.39,
        433.57,  433.54,  434.73,  435.28,  434.81,  429.97,  430.18,
        431.6 ,  433.72,  431.21,  434.62,  434.65,  432.79,  434.46,
        426.95,  426.29,  428.67,  428.93,  430.37,  430.21,  432.14,
        433.07,  440.08,  441.53,  430.2 ,  428.6 ,  432.24,  424.35,
        422.93,  423.03,  426.12,  425.64,  423.66,  428.52,  431.68,
        427.33,  423.89,  423.2 ,  423.28,  431.6 ,  430.52,  430.25,
        435.75,  442.37,  442.88,  442.91,  429.51,  433.2 ,  431.33,
        439.78,  443.67,  447.14,  426.03,  408.84,  410.49,  405.05,
        401.  ,  401.88,  406.37,  404.3 ,  402.81,  411.98,  411.29,
        410.7 ,  405.8 ,  406.26,  401.46,  403.27,  397.97,  394.92,
        406.33,  398.68,  402.56,  403.04,  400.54,  399.72,  405.88,
        399.4 ,  394.66,  389.89,  384.31,  391.4 ,  384.04,  387.3 ,
        382.99,  390.31,  383.85,  385.43, -999.9 ,  380.07,  380.74,
        386.94,  390.33,  391.44], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2019-04-06 ... 2019-04-08T23:55:00
Attributes:
    CDF:           {'VATT': {'FIELDNAM': 'Solar Wind Proton Speed', 'VALIDMIN...
    data_att:      {'coord_sys': '', 'units': 'km/s', 'depend_1_units': 'km/s...
    plot_options:  {'xaxis_opt': {'axis_label': 'Time', 'crosshair': 'X', 'x_...